In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from itertools import cycle
from matplotlib import pyplot as plt
from kneed import KneeLocator

In [ ]:
w_data_path = ("../1_data/setup/wip_metrics_fjc(100).csv")
wjm_data_path = ("../1_data/setup/wip_metrics_jm(100).csv")
plot_base = "../1_data/plots"

wdf = pd.read_csv(w_data_path, index_col=0)
wdf_jm = pd.read_csv(wjm_data_path, index_col=0)

In [ ]:
def get_balanced_wip_size(wip_metrics_df, verbose=False):
    wdf_g = wip_metrics_df.groupby('wip_size').mean()
    err_wdf = np.argmin((wdf_g['flow_time_rel_d_ave'] - wdf_g['ave_utilization']).abs())
    best_wip = wdf_g.iloc[err_wdf,].name
    if verbose:
        print(f"Best wip size: {best_wip}")
        display(wdf_g.iloc[err_wdf,])
    return best_wip, wdf_g.iloc[err_wdf,]['flow_time_rel_d_ave']
    
get_balanced_wip_size(wdf, True)
get_balanced_wip_size(wdf_jm, True)

In [ ]:
%%time

def plot_characteristic_curves(data, ax, color, problem, right=False):
    ax = sns.lineplot(
        data=data, x='wip_size', y='ave_utilization', 
        label='Resource\nUtilization', color=next(color), ax=ax)
#     ax = sns.lineplot(
#         data=data, x='wip_size', y='average_flow_time', 
#         label='Flow Time', color=next(color), ax=ax)
    ax2 = ax.twinx()
    ax.set_ylabel('Utilization')
    ax.set_xlabel("WIP Size")
    ax.set_title(f"{problem} Characteristic Curves")
    ax.get_legend().remove()
    
    
    ax2 = sns.lineplot(
        data=data, x='wip_size', y=data['flow_time_rel_d_ave'], 
        label='Job Duration\n(Relative to\nFlow Time)', color=next(color), ax=ax2)
    ax2.set_ylabel('Relative Job Duration')
    ax2.get_legend().remove()
    x, y = get_balanced_wip_size(data)
    print(x, y)
    ax2.annotate(f'WIP: {x}', xy=(x, y), xycoords='data',
        xytext=(40, 30), textcoords='offset points',
        arrowprops=dict(arrowstyle="->", connectionstyle="angle3,angleA=0,angleB=90", color='black'))
    if not right:
        ax.figure.legend(ncol=1, loc='center right', bbox_to_anchor=(0.9, 0.5))

sns.set_style('whitegrid')
sns.set_context('talk')
color = cycle(sns.color_palette('cubehelix', 2))

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_characteristic_curves(wdf_jm, axes[0], color, "$Jm$")
plot_characteristic_curves(wdf, axes[1], color, "$FJc$", True)

fig.subplots_adjust(right=0.5)
plt.tight_layout()


plt.savefig(f"{plot_base}/51_wip_metrics_fjc&jm.png", dpi=200, bbox_inches='tight', pad_inches=0.02)
plt.show()

In [ ]:
%%time

# ! pip install kneed
def plot_elbow_comparison(data, problem='fjc'):
    sns.set_style('whitegrid')
    sns.set_context('talk')
    color = cycle(sns.color_palette('cubehelix', 4))
    
    df = data.copy()
    df['throughput'] = df['wip_size'] / df['t']
    df['flow_time'] = df['cmax_rel_flow_time'] * df['t']
    
    data_fjc_means = df.groupby('wip_size').mean().reset_index(drop=False)
    data_fjc_std = df.groupby('wip_size').std().reset_index(drop=False)

    xs=(data_fjc_means['cmax_rel_flow_time'] * data_fjc_means['t']) 
    ys=(data_fjc_means['wip_size'] / data_fjc_means['t']) 

    ax = sns.lineplot(
        data=data_fjc_means, 
        x='flow_time', y='throughput', 
        label='Characteristic Curve', color=next(color))
    ax2 = ax.twiny()
    ax2 = sns.lineplot(
          data=data_fjc_means, 
          x='wip_size', y='throughput', 
          label=None, alpha=.0)
    ax.fill_between(data_fjc_means['flow_time'], 
                    data_fjc_means['throughput'] - data_fjc_std['throughput'], 
                    data_fjc_means['throughput'] + data_fjc_std['throughput'], 
                    alpha=.5, color=next(color))


    el = KneeLocator(
        x=data_fjc_means['flow_time'],
        y=data_fjc_means['throughput'],
        curve='concave',
        direction='increasing',
        interp_method='polynomial',
    )
    wip1 = data_fjc_means[data_fjc_means['flow_time'] == el.elbow]['wip_size'].values[0]
    ax = sns.scatterplot(x=[el.elbow], y=[el.elbow_y], color=next(color), label=f'Elbow Heuristic WIP: {wip1}', ax=ax)
    wip2, _ = get_balanced_wip_size(data_fjc_means)
    el_row = data_fjc_means[data_fjc_means['wip_size'] == wip2]
    el_x = ((el_row['flow_time'])).values[0]
    el_y = ((el_row['throughput'])).values[0]
    ax = sns.scatterplot(x=[el_x], y=[el_y], color=next(color), label=f'Chosen WIP Size: {wip2}', ax=ax)

    ax.set_ylabel('Throughput (Jobs/Time Unit)')
    ax.set_xlabel("Flow Time (Time Unit)")
    ax2.set_xlabel("WIP Size")
    plt.tight_layout()
    plt.savefig(f"{plot_base}/51_wip_elbow_cmp_{problem.lower()}.png", dpi=200, bbox_inches='tight', pad_inches=0.02)
    plt.show()

    
plt.figure(figsize=(5.5, 4.5))
print("Relationship between the elbow heuristic and ours -- Jm:")
plot_elbow_comparison(wdf_jm, problem='jm')
print("Relationship between the elbow heuristic and ours -- FJc:")

plt.figure(figsize=(6, 4.5))
plot_elbow_comparison(wdf)

# ax.annotate(f'WIP: 14', xy=(el_x, el_y), xycoords='data',
#     xytext=(40, 30), textcoords='offset points',
#     arrowprops=dict(arrowstyle="->", connectionstyle="angle3,angleA=0,angleB=90", color='black'))